In [3]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import random
import oracledb
from dotenv import load_dotenv
import os 

- 상품 설명이 생각보다 길어서 varchar(500)에서 varchar(800)으로 늘림

- 1:1문의 내역에 관리자 답변 컬럼 안넣어서 추가함
- 1:1문의 -> 문의처럼 띄어쓰기, 특수문자 테이블명 제거 수행
	- 이걸 좀 더 일찍 알았어야 했다

- 쿠폰 시작기간은 있는데 왜 만료기간은 없냐..
	- 만료날짜 추가
	- 50000원 이상 사면 20% 할인 이렇게 하고 싶은데 이건 어케 설계하는지 모름

- 상품문의에 관리자 ID, 관리자답변 까묵어서 추가함
	- ㅏ아앙ㅇ아ㅏ아앙 답변상태 안넣었어...

- 결제 테이블
	- 총 결제금액 -> 총 상품금액(배송비제외)
	- 추가사용가능쿠폰ID -> 사용가능쿠폰ID
	- 사용가능쿠폰 할인율, 할인금액 추가하기

In [4]:
load_dotenv('.env')
user_name = os.getenv("user_name")
password = os.getenv('password')
dsn = os.getenv('dsn')

In [5]:
try:
	oracledb.init_oracle_client(lib_dir=r"C:\instant_client\instantclient_21_19")
except:
    oracledb.init_oracle_client(lib_dir=r"D:\instantclient_23_9")

conn = oracledb.connect(
    user=user_name,          # 사용자명
    password=password,      # 비밀번호
    dsn=dsn # 접속 정보 (SQL Developer와 동일)
)
cur = conn.cursor()

In [3]:
# 시퀀스 값 찾기
def return_sequence_value(sequence_name):
	query = f'select {sequence_name}.nextval from dual'
	sequence = pd.read_sql(query, conn)
	return sequence.iat[0, 0]

def insert_data(table_name, db_col, df):
    columns = ', '.join(db_col)
    placeholders = ', '.join([f':{k}' for k in db_col])

    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    
    data = df.to_dict("records")

    cur.executemany(sql, data)
    conn.commit()

def random_date2025():
    start_date = datetime(2025, 1, 1)
    end_date = datetime(2025, 12, 31)

    delta_days = (end_date - start_date).days

    random_date = start_date + timedelta(days=random.randint(0, delta_days))
    return random_date

#### 1. 회원 테이블 데이터 채우기 (99,950건)

In [17]:
user = pd.read_csv('데이터_회원테이블.csv')

user['생년월일'] = pd.to_datetime(user['생년월일'])
user['탈퇴날짜'] = None
user['탈퇴여부'] = None

data = user.to_dict("records")

insert_data('회원', user.columns, user)

#### 2. 상품 테이블 데이터 채우기 (50,000건)

In [18]:
product = pd.read_csv('데이터_상품테이블_50000.csv')
product.columns = [i.replace(' ','') for i in product.columns]
insert_data('상품', product.columns, product)

#### 3. 관리자 테이블 데이터 채우기 (50건)

In [19]:
manager = pd.read_csv('데이터_관리자테이블.csv')
manager['계정생성일'] = pd.to_datetime(manager['계정생성일'])
manager['최근로그인시간'] = pd.to_datetime(manager['최근로그인시간'])

insert_data('관리자', list(manager.columns), manager)

#### 4. 재고 테이블 채우기 (50,000건)

In [37]:
stock = pd.read_csv('데이터_재고테이블.csv')
stock['마지막입고일'] = pd.to_datetime(stock['마지막입고일'])
stock['입고예상일'] = pd.to_datetime(stock['입고예상일'])

insert_data('재고', list(stock.columns), stock)

#### 5. 최근본상품 테이블 데이터 채우기 (100,000건)

In [12]:
recently_see = pd.read_csv('데이터_최근본상품테이블.csv')
recently_see['조회일자'] = pd.to_datetime(recently_see['조회일자'])

insert_data('최근본상품', list(recently_see.columns), recently_see)

#### 6. 개인문의 테이블 데이터 채우기 (150,000건)

In [ ]:
pq = pd.read_csv('데이터_개인문의테이블.csv')
pq['작성일'] = pd.to_datetime(pq['작성일'])
pq = pq.where(pd.notnull(pq), None)

insert_data('개인문의', list(pq.columns), pq)

#### 7. 쿠폰 테이블 데이터 채우기 (300건)

In [5]:
manager = pd.read_sql("select 관리자ID from 관리자 where 권한 = 'SUPER_ADMIN'", conn)
coupon = pd.read_sql("select * from 쿠폰", conn)

datas = []

for i in range(1,301):
	discount = random.choice([
		[None, int(random.choice(range(1000, 10000, 1000)))],  # 1000~9000원
		[int(random.randint(5, 90)), None]                     # 5~90%
	])

	start_day = random_date2025()
	remaining_days = (datetime(2025, 12, 31) - start_day).days
	end_day = start_day + timedelta(days = random.randint(remaining_days//2,remaining_days))

	if discount[0] != None: # 할인율 단위
		data = [
		i, # 4번 이후부터는 sequence로 바꿔야 함 
		int(random.choice(manager['관리자ID'])),
		f'{discount[0]}% 할인 쿠폰', start_day, end_day,
		] + discount
	else:
		data = [
		i, # 4번 이후부터는 sequence로 바꿔야 함 
		int(random.choice(manager['관리자ID'])),
		f'{discount[1]}원 할인 쿠폰', start_day, end_day,
		] + discount
	datas.append(data)

coupon_csv = pd.DataFrame(datas)
coupon_csv.columns = coupon.columns 
coupon_csv.to_csv('데이터_쿠폰테이블.csv',index=False)

C:\Users\amysm\AppData\Local\Temp\ipykernel_28072\886554220.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  manager = pd.read_sql("select 관리자ID from 관리자 where 권한 = 'SUPER_ADMIN'", conn)
C:\Users\amysm\AppData\Local\Temp\ipykernel_28072\886554220.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  coupon = pd.read_sql("select * from 쿠폰", conn)


In [6]:
coupon = pd.read_csv('데이터_쿠폰테이블.csv')
coupon['시작날짜'] = pd.to_datetime(coupon['시작날짜'])
coupon['만료날짜'] = pd.to_datetime(coupon['만료날짜'])
coupon = coupon.where(pd.notnull(coupon), None)

for c in coupon.select_dtypes('float64').columns:
    coupon[c] = coupon[c].astype('Int64')  

insert_data('쿠폰', list(coupon.columns), coupon)

#### 8. 이벤트 테이블 데이터 채우기 (20건)

In [ ]:
manager = pd.read_sql("select 관리자ID from 관리자 where 권한 = 'CONTENT_MANAGE'", conn)
event = pd.read_sql('select * from 이벤트', conn)
coupon = pd.read_sql('select * from 쿠폰', conn)

event_name = ['쨌든 할인', '암튼 할인' ,'왜 안사?']
event_content = ['쨌든 사라', '암튼 사라', '왜 안사?']

datas = []

for i in range(1,20):
    data = [
        i, # 3번 이후부터는 sequence로 바꿔야 함
		int(random.choice(manager['관리자ID'])),
		random.choice(event_name) + str(i),
		coupon['시작날짜'].iloc[i-1],
		coupon['만료날짜'].iloc[i-1],
		random.choice(event_content) + str(i)
	]
    datas.append(data)

event_csv = pd.DataFrame(datas)
event_csv.columns = event.columns 
event_csv.to_csv('데이터_이벤트테이블.csv',index=False)

C:\Users\amysm\AppData\Local\Temp\ipykernel_13408\843098096.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  manager = pd.read_sql("select 관리자ID from 관리자 where 권한 = 'CONTENT_MANAGE'", conn)
C:\Users\amysm\AppData\Local\Temp\ipykernel_13408\843098096.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  event = pd.read_sql('select * from 이벤트', conn)
C:\Users\amysm\AppData\Local\Temp\ipykernel_13408\843098096.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  coupon = pd.read_sql('select * from 쿠폰', conn)


In [25]:
event = pd.read_csv('데이터_이벤트테이블.csv')
event['이벤트시작일'] = pd.to_datetime(event['이벤트시작일'])
event['이벤트종료일'] = pd.to_datetime(event['이벤트종료일'])

insert_data('이벤트', list(event.columns), event)

#### 9. 장바구니 테이블 데이터 채우기 (100,000건)

- 장바구니 담을 때 최대 재고수량까지만 담을 수 있게 함
- 이 때 다른 유저들이 담아둔 상품 개수는 신경쓰지 않음

In [26]:
cart = pd.read_csv('데이터_장바구니테이블.csv')
cart['추가날짜'] = pd.to_datetime(cart['추가날짜'])
cart = cart.where(pd.notnull(cart), None)

insert_data('장바구니', list(cart.columns), cart)

#### 10. 상품문의 테이블 데이터 채우기 (150,000건)

In [36]:
product_q = pd.read_csv('데이터_상품문의테이블.csv')
product_q = product_q.where(pd.notnull(product_q), None)
product_q['관리자ID'] = product_q['관리자ID'].astype('Int64') 
    
insert_data('상품문의', list(product_q.columns), product_q)

#### 11. 쿠폰이력 테이블 데이터 채우기 (983,300건)

- 쿠폰 ID, 회원 ID 이렇게 세트는 중복 불가능
	- 원래 100만 건이었는데 중복 지워서 저렇게 됨

In [5]:
coupon_record = pd.read_csv('데이터_쿠폰이력테이블.csv')
coupon_record = coupon_record.drop_duplicates(['쿠폰ID','회원ID'])

insert_data('쿠폰이력', list(coupon_record.columns), coupon_record)

#### 12. 주문 테이블 데이터 채우기 - 정상 주문/결제 완료 (3,000,000건)

- 주문상태 : 주문완료, 주문취소, 환불완료
- 포인트 변동 사유 : 사용, 적립, 소멸
	- 소멸까지 들어가면 일이 복잡해져서 일단 사용으로 통일

- 어쩌다보니 결제seq랑 주문 seq랑 동일해졌는데 실제로 데이터 더 넣다보면 또 달라질 수 있음


In [8]:
order = pd.read_csv('데이터_주문테이블.csv')
order['주문날짜'] = pd.to_datetime(order['주문날짜'])
insert_data('주문', list(order.columns), order)

#### 13. 포인트 테이블 데이터 채우기 (350,000건)

In [55]:
point = pd.read_csv('데이터_포인트테이블.csv')
point['변동날짜'] = pd.to_datetime(point['변동날짜'])
insert_data('포인트', list(point.columns), point)

#### 14. 주문상세 테이블 데이터 채우기 (7,000,000건)

- 주문 상세에 따른 재고수량 변동 존재

In [ ]:
order_detail = pd.read_csv('데이터_주문상세테이블.csv')
insert_data('주문상세', list(order_detail.columns), order_detail)

(7000000, 5)

#### 15. 배송 테이블 데이터 채우기 (3,000,000건)

- 주문 ID에서 중복 발생
	- records = [
    {'seq': i + 1, **row}
    for i, row in enumerate(condi_df.to_dict('records'))] 이렇게 고정시켜서 집어넣음

In [14]:
import pandas as pd 
delivery = pd.read_csv('데이터_배송테이블.csv')
delivery = delivery.where(pd.notnull(delivery), None)
insert_data('배송', list(delivery.columns), delivery)

#### 16. 결제 테이블 데이터 채우기 (3,000,000건)

- 결제수단 : 무통장입금, 카드 결제 이런거도 나중에 넣으면 좋을듯

In [ ]:
comb_df = pd.read_sql('''SELECT
    o.결제ID,
    o.주문ID,
    '결제완료' AS 결제상태,
    o.주문날짜
        + NUMTODSINTERVAL(TRUNC(DBMS_RANDOM.VALUE(10, 20)), 'SECOND')
        AS 결제날짜,
    CASE MOD(o.주문ID, 3)
        WHEN 0 THEN '네이버페이'
        WHEN 1 THEN '카카오페이'
        ELSE '토스페이'
    END AS 결제수단,

    /* 상품금액 */
    NVL(
        SUM(p.원가 * ((100 - p.할인율) / 100) * od.주문수량),
        0
    ) - NVL(MAX(o.사용포인트), 0) AS 총상품금액,

    /* 배송비 */
    CASE
        WHEN NVL(SUM(p.원가 * ((100 - p.할인율) / 100) * od.주문수량), 0) >= 30000
            THEN 0
        ELSE 3000
    END AS 배송비,

    c.쿠폰ID as 사용가능쿠폰ID,
    c.할인율 as 적용쿠폰할인율,
    c.할인금액 as 적용쿠폰할인금액
FROM 주문 o
    LEFT JOIN 주문상세 od ON o.주문ID = od.주문ID
    LEFT JOIN 상품 p ON od.상품ID = p.상품ID
    LEFT JOIN (
        SELECT
            ch.회원ID,
            ch.쿠폰ID,
            c.할인율,
            c.할인금액,
            c.시작날짜,
            c.만료날짜,
            ROW_NUMBER() OVER (
                PARTITION BY ch.회원ID
                ORDER BY DBMS_RANDOM.VALUE
            ) rn
        FROM 쿠폰이력 ch
            JOIN 쿠폰 c ON ch.쿠폰ID = c.쿠폰ID
        WHERE ch.사용여부 = 'N'
    ) c
        ON o.회원ID = c.회원ID
        AND c.rn = 1
        AND o.주문날짜 BETWEEN c.시작날짜 AND c.만료날짜
GROUP BY
    o.결제ID,
    o.주문ID,
    o.주문날짜,
    o.회원ID,
    c.쿠폰ID,
    c.할인율,
    c.할인금액''', conn)

comb_df.to_csv('데이터_결제테이블.csv',index=False)

In [18]:
pay = pd.read_csv('데이터_결제테이블.csv')
pay['결제날짜'] = pd.to_datetime(pay['결제날짜'])
pay[['사용가능쿠폰ID','적용쿠폰할인율', '적용쿠폰할인금액']] = pay[['사용가능쿠폰ID','적용쿠폰할인율', '적용쿠폰할인금액']].astype('Int64')
pay = pay.where(pd.notnull(pay), None)
insert_data('결제', list(pay.columns), pay)

In [19]:
pay.shape

(3000000, 10)

#### 17. 리뷰 테이블 데이터 채우기 (500,000건)

- 사용자가 주문한 상품만 리뷰 쓸 수 있게

In [9]:
review = pd.read_csv('데이터_리뷰테이블.csv')
review['작성날짜'] = pd.to_datetime(review['작성날짜'])
review = review.where(pd.notnull(review), None)
insert_data('리뷰', list(review.columns), review)